In [1]:
from spacerocks import SpaceRock
from spacerocks.spice import SpiceKernel
from spacerocks.time import Time
from spacerocks.nbody import Simulation, Integrator, Force

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np

kernel = SpiceKernel()
kernel.load("/Users/kjnapier/data/spice/latest_leapseconds.tls")
kernel.load("/Users/kjnapier/data/spice/de440s.bsp")

In [2]:
ARCSEC_PER_RAD = 206264.80624709636

In [3]:
epoch = Time.now()
planets_names = ["sun", "mercury barycenter"]
planets = [SpaceRock.from_spice(name, epoch, reference_plane="ECLIPJ2000", origin='ssb') for name in planets_names]

In [4]:
sim = Simulation()

sim.set_epoch(epoch)
sim.set_origin("ssb")
sim.set_reference_plane("ECLIPJ2000")
sim.set_integrator(Integrator.ias15(timestep=1.0))
sim.add_force(Force.solar_gr())

print(sim.origin)

for planet in planets:
    sim.add(planet)
    
sim.move_to_center_of_mass()

print(sim.origin)

Origin: SSB with mu = 0.0002963092749396808
Origin: simulation_barycenter with mu = 0.0002959122574091215


In [5]:
mercury = sim.get_particle("mercury barycenter")
evec_0 = np.array(mercury.evec) / np.linalg.norm(mercury.evec)

In [6]:
n = 10
sim.integrate(epoch + n * 100 * 365.25)

thread '<unnamed>' panicked at /Users/kjnapier/spacerocks/src/nbody/forces/gravity.rs:66:52:
index out of bounds: the len is 2 but the index is 27
note: run with `RUST_BACKTRACE=1` environment variable to display a backtrace


PanicException: index out of bounds: the len is 2 but the index is 27

In [7]:
mercury = sim.get_particle("mercury barycenter")
evec_1 = np.array(mercury.evec) / np.linalg.norm(mercury.evec)

In [8]:
precession_rate = np.arccos(np.dot(evec_0, evec_1)) * ARCSEC_PER_RAD / n

print(f"Mercury's pericenter precession rate due to GR is {precession_rate:.2f} arcsec/century.")

Mercury's pericenter precession rate due to GR is 0.00 arcsec/century.
